# 1) Data Prep

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

0 C:\Users\p.schambach\Desktop\DSR\drone_steering\models
1 C:\Users\p.schambach\Desktop\DSR\drone_steering


## a) Data loading

In [5]:
from app_local.module import DataEnsembler, GestureTransformer

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.load_data()

## b) Data rescaling to actual video length

## c) Interpolate and adjust framebased labels

## d) Generate training data

In [6]:
de.assemble_data(tolerance_range=2)

In [7]:
de.display_information()

i: 0 	shape X: (603, 18, 16) 	shape y: (603,) 	count: 101
i: 1 	shape X: (519, 18, 16) 	shape y: (519,) 	count: 89
i: 2 	shape X: (498, 18, 16) 	shape y: (498,) 	count: 119
i: 3 	shape X: (650, 18, 16) 	shape y: (650,) 	count: 230
i: 4 	shape X: (508, 18, 16) 	shape y: (508,) 	count: 147
i: 5 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 146
i: 6 	shape X: (601, 18, 16) 	shape y: (601,) 	count: 159
i: 7 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 127
i: 8 	shape X: (490, 18, 16) 	shape y: (490,) 	count: 110
i: 9 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 118
i: 10 	shape X: (492, 18, 16) 	shape y: (492,) 	count: 121
i: 11 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 87
i: 12 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 126
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 127
i: 14 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 114
i: 15 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 115
i: 16 	shape X: (541, 18, 16) 	shape y: (541,) 	count: 92
i: 17 	sha

## e) Data normalization

In [8]:
X = de.X.copy()
y = de.y.copy().astype("int32")
gt = GestureTransformer(byrow=True, feature_names= list(de.feature_names))
X = gt.transform(X, verbose = True)

---------------------------------------------------------
Performing row-based transformations:
X-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (18188, 18)
Y-Mean of Hip and Shoulders:
	Row-based Mean is taken over each sample and each timestep
	Shape: (18188, 18)
Scaling based on Hip-Shoulder distance:
	Row-based distance is calculated for each sample and each timestep
	Shape: (18188, 18)
-----------------------------------------------------
X-Shift:
            min   mean  median    max   std
timestep                                   
0         196.8  294.5   280.8  511.9  38.9
1         196.8  294.4   280.7  511.9  38.9
2         196.8  294.4   280.7  511.9  38.9
3         196.8  294.3   280.6  511.9  38.9
4         196.8  294.3   280.6  511.9  38.9
5         196.8  294.3   280.5  511.9  38.9
6         196.8  294.2   280.5  511.9  38.9
7         196.8  294.2   280.4  511.9  38.9
8         196.8  294.1   280.4  511.9  38.9
9         1

# 2) Model Training

## a) Filtering 0-Labels

In [13]:
from app_local.module import DataResampler
dr_var = DataResampler()
dr_var.fit(X,y)
dr_var.restrict_zero_labels(500, criterion="variance_lowest")
dr_var.display_information()

-------------------------------------
Current number of zero-labels: 13894
New number of zero-labels (selected with criterion="variance_lowest"): 500


,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,500,10.43
1,701,3.85,701,14.62
2,576,3.17,576,12.02
3,991,5.45,991,20.67
4,677,3.72,677,14.12
5,660,3.63,660,13.77
6,689,3.79,689,14.37


In [14]:
dr = DataResampler()
dr.fit(X,y)
dr.restrict_zero_labels(500)
dr.display_information()

-------------------------------------
Current number of zero-labels: 13894
New number of zero-labels: 500


,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,500,10.43
1,701,3.85,701,14.62
2,576,3.17,576,12.02
3,991,5.45,991,20.67
4,677,3.72,677,14.12
5,660,3.63,660,13.77
6,689,3.79,689,14.37


In [27]:
print(dr.X.shape)
X_full_zero = np.concatenate( [dr.X, dr_var.X[(dr_var.y == 0),:,:]] )
y_full_zero = np.concatenate( [dr.y, dr_var.y[dr_var.y == 0]])
print(X_full_zero.shape, y_full_zero.shape)

(4794, 18, 16)
(5294, 18, 16) (5294,)


In [28]:
dr_up = DataResampler()
dr_up.fit(X_full_zero, y_full_zero)
dr_up.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,1000,18.89,1000,18.89
1,701,13.24,701,13.24
2,576,10.88,576,10.88
3,991,18.72,991,18.72
4,677,12.79,677,12.79
5,660,12.47,660,12.47
6,689,13.01,689,13.01


In [30]:
dr_up.upsample(n = 800, exceptions=[0,3])
dr_up.display_information()
X_final = dr_up.X
y_final = dr_up.y

## b) OneHot-Encoding of target vector

In [31]:
from sklearn.preprocessing import OneHotEncoder

In [32]:
ohe = OneHotEncoder(sparse = False)
print(type(ohe))
target = ohe.fit_transform(y_final.reshape(-1,1))
print("Before:", y_final.shape)
print("After:", target.shape)
target[0:5,:]

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>
Before: (5991,)
After: (5991, 7)


array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.]])

## c) Train-Test-Split

## d) Model Training

In [34]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### Train a simple SV-Classifier

In [41]:
x_final = X_final.reshape(X_final.shape[0],-1)

model = SVC(
    random_state = 42,
    gamma = "auto"
)

model.fit(x_final, y_final)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [46]:
preds = model.predict(x_final)
(preds == y_final).sum()/len(y_final)

0.9512602236688366

### Gridsearch with Crossvalidation

In [47]:
model.get_params().keys()

dict_keys(['C', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

In [48]:
# Number of trees in random forest
n_estimators = [100, 500, 1000]
# Number of features to consider at every split
max_features = [5, 10, 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 4)]
max_depth.append(None)

# Create the random grid
random_grid = {'randomforestclassifier__n_estimators': n_estimators,
               'randomforestclassifier__max_features': max_features,
               'randomforestclassifier__max_depth': max_depth,
              }

random_grid = {
                'C': [10,5.0.,2.0,1.0, 0.5, 0.1, 0.5, 0.01]
              }

print(random_grid)

{'C': [1.0, 0.5, 0.1, 0.5, 0.01]}


#### Training takes quite a long time - do not needlessly repeat it

In [49]:
params = random_grid
scoring = "accuracy"
clf = GridSearchCV(model, params, n_jobs = -1, verbose = 10, scoring = scoring)
clf.fit(x_final, y_final)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  15 | elapsed:   13.3s remaining:  1.4min
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:   15.3s remaining:   42.3s
[Parallel(n_jobs=-1)]: Done   6 out of  15 | elapsed:   17.1s remaining:   25.7s
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:   24.2s remaining:   21.2s
[Parallel(n_jobs=-1)]: Done  10 out of  15 | elapsed:   24.9s remaining:   12.4s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   28.6s remaining:    7.1s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   43.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': [1.0, 0.5, 0.1, 0.5, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=10)

In [50]:
clf.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'mean_train_score', 'std_train_score'])

In [51]:
clf.cv_results_["mean_test_score"]

array([0.92505425, 0.91687531, 0.89467535, 0.91687531, 0.37656485])

In [53]:
#from sklearn.externals import joblib
#joblib.dump(clf,'model/model_geture_svm_nip120_[1000,800]_00x.pkl')

['model_geture_svm_[1000,800]_001.pkl']

In [54]:
#loaded_model = joblib.load('')
#loaded_model.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'mean_train_score', 'std_train_score'])

In [8]:
import os
import re

file_names = os.listdir('models/')
print(file_names)

['.ipynb_checkpoints', '.pytest_cache', 'model_gesture_ip.ipynb', 'model_gesture_lstm_ip.ipynb', 'model_gesture_lstm_ip50_tr0.h5', 'model_gesture_lstm_ip50_tr2_sazero1000.h5', 'model_gesture_lstm_nip120.ipynb', 'model_gesture_lstm_nip120_tr0_sazero0.h5', 'model_gesture_lstm_nip120_tr0_sazero1000.h5', 'model_gesture_lstm_nip120_tr0_sazero500.h5', 'model_gesture_lstm_nip120_tr2_sa4000.h5', 'model_gesture_lstm_nip120_tr2_sa5000.h5', 'model_gesture_lstm_nip120_tr2_saex500.h5', 'model_gesture_lstm_nip120_tr2_sazero0.h5', 'model_gesture_lstm_nip120_tr2_sazero1000.h5', 'model_gesture_lstm_nip120_tr2_sazero500.h5', 'model_gesture_lstm_nip120_tr4_sazero0.h5', 'model_gesture_lstm_nip120_tr4_sazero1000.h5', 'model_gesture_lstm_nip120_tr4_sazero500.h5', 'model_gesture_lstm_nip120_tr6_sazero0.h5', 'model_gesture_lstm_nip120_tr6_sazero1000.h5', 'model_gesture_lstm_nip120_tr6_sazero500.h5', 'model_gesture_split.ipynb', 'model_gesture_svm_nip120.ipynb', 'model_gesture_svm_nip120_[1000,800]_001.pkl', '

In [29]:
test_str = 'aaaaa_bbbbb_cccc_dddd'
pattern = '(?P<a>)_(?P<not_>[^_]*).*'
r = re.compile(pattern)
match = r.search(test_str)
match.groupdict()

{'a': '', 'not_': 'bbbbb'}

In [12]:
'model_gesture_svm_nip120_[1000,800]_001.pkl'
model_type = "gesture"
interpolation = "nip"
pattern = '(?P<model_type>model_' + model_type + '{1}_.*)(?P<ip_nip>' + interpolation\
            + '{1})(?P<model_parameters>.+).(?P<suffix>pkl|h5)'

In [5]:
import re
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)
    
model_type = 'gesture'
interpolation = 'nip'
'-----------------------------------------------------------------------'

file_names = os.listdir('models/')

grp_model_type = '(?P<model_type>model_'+ model_type + '_[^_]+)'
grp_ip = '(?P<ip_nip>' + interpolation + '\d*)'
grp_model_params = '(?P<model_parameters>.+)'
grp_suffix = '(?P<suffix>pkl|h5)'
pattern = '(?P<file_name>' + grp_model_type + '_' + grp_ip + '_' + grp_model_params + '.' + grp_suffix + ')'


reg = re.compile(pattern)

matches = []
for file_name in file_names:
    match = reg.search(file_name)
    if match:
        matches.append(match)

groups = []
for i, match in enumerate(matches):
    group = match.groupdict()
    groups.append(group)


models = []
for grp in groups:
    models.append(grp["file_name"])



print('Which model instance do you want to use?')
for i in range(len(models)):
    print(str(i) + ' | ' + models[i])
#model_instance = input()
print('')



0 C:\Users\p.schambach\Desktop\DSR\drone_steering
Which model instance do you want to use?
0 | model_gesture_lstm_nip120_tr0_sazero0.h5
1 | model_gesture_lstm_nip120_tr0_sazero1000.h5
2 | model_gesture_lstm_nip120_tr0_sazero500.h5
3 | model_gesture_lstm_nip120_tr2_sa4000.h5
4 | model_gesture_lstm_nip120_tr2_sa5000.h5
5 | model_gesture_lstm_nip120_tr2_saex500.h5
6 | model_gesture_lstm_nip120_tr2_sazero0.h5
7 | model_gesture_lstm_nip120_tr2_sazero1000.h5
8 | model_gesture_lstm_nip120_tr2_sazero500.h5
9 | model_gesture_lstm_nip120_tr4_sazero0.h5
10 | model_gesture_lstm_nip120_tr4_sazero1000.h5
11 | model_gesture_lstm_nip120_tr4_sazero500.h5
12 | model_gesture_lstm_nip120_tr6_sazero0.h5
13 | model_gesture_lstm_nip120_tr6_sazero1000.h5
14 | model_gesture_lstm_nip120_tr6_sazero500.h5
15 | model_gesture_svm_nip120_[1000,800]_001.pkl



In [10]:
.append(model['model_type'] + model['ip_nip'] + model['model_parameters'])

In [8]:
models


['model_gesture_lstm_nip120_tr0_sazero0',
 'model_gesture_lstm_nip120_tr0_sazero1000',
 'model_gesture_lstm_nip120_tr0_sazero500',
 'model_gesture_lstm_nip120_tr2_sa4000',
 'model_gesture_lstm_nip120_tr2_sa5000',
 'model_gesture_lstm_nip120_tr2_saex500',
 'model_gesture_lstm_nip120_tr2_sazero0',
 'model_gesture_lstm_nip120_tr2_sazero1000',
 'model_gesture_lstm_nip120_tr2_sazero500',
 'model_gesture_lstm_nip120_tr4_sazero0',
 'model_gesture_lstm_nip120_tr4_sazero1000',
 'model_gesture_lstm_nip120_tr4_sazero500',
 'model_gesture_lstm_nip120_tr6_sazero0',
 'model_gesture_lstm_nip120_tr6_sazero1000',
 'model_gesture_lstm_nip120_tr6_sazero500']